In [1]:
# Load CSV using Pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
filename = 'LRC_dataset_008.csv'
names = ['sample_no','current_a', 'voltage_v','position_deg','velocity_degs',
          'setpoint_deg','torqsens_nm', 'disturbance_nm','collision']
data = pd.read_csv(filename, names=names)
data.drop(['sample_no'], axis=1) #remove for now
data[1:10]

IOError: File LRC_dataset_008.csv does not exist

In [ ]:
#Balance the dataset

from sklearn.utils import resample

def data_downsampling(data):

    # Separate majority and minority classes
    data_majority = data[data.collision==-1]
    data_minority = data[data.collision==+1]


    majority_count = data_majority.shape[0]
    minority_count = data_minority.shape[0]

    # Downsample the majority class
    data_majority_downsampled = resample(data_majority, 
                                 replace=False,     # sample with replacement
                                 n_samples=minority_count,    # to match minority class
                                 random_state=123) # reproducible results

    new_majority_count = data_majority_downsampled.shape[0]
 
    # Combine the minority class with the downsampled majority class
    data_downsampled = pd.concat([data_minority, data_majority_downsampled])
    
    dataset_balance = float(new_majority_count) / data_downsampled.shape[0]

    print(dataset_balance)

    print(type(data_downsampled))

    data = data_downsampled

    return data

In [ ]:
data = data_downsampling(data)

In [ ]:
#Normalization
from sklearn.preprocessing import MinMaxScaler
features = ['current_a', 'voltage_v','position_deg','velocity_degs',
          'setpoint_deg','torqsens_nm']

scaler = MinMaxScaler()

data[features] = scaler.fit_transform(data[features])

data[1:10]

In [ ]:
#Split test-train data
train_data=data.sample(frac=0.8,random_state=200)
test_data=data.drop(train_data.index)

print(train_data.shape[0]/float(train_data.shape[0] + test_data.shape[0]))

In [ ]:
#convert the dataframe into useful arrays
def get_panda_data(dataframe, features, label):
    dataframe['constant'] = 1 #w0
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = features_frame.as_matrix()
    label_sarray = dataframe[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

In [ ]:
features = ['current_a', 'voltage_v','position_deg','velocity_degs',
          'setpoint_deg','torqsens_nm']
label = 'collision'

[train_feature_matrix, train_collision] = get_panda_data(train_data, features, label)
[test_feature_matrix, test_collision] = get_panda_data(test_data, features, label)

print(train_feature_matrix.shape[0]+test_feature_matrix.shape[0])
print(data.shape[0])

In [ ]:
#Generate Initial Coefficients
initial_coefficients = train_feature_matrix[1,:]
initial_coefficients.fill(0) #set coefficients to zero
initial_coefficients.size

In [ ]:
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    score = np.dot(feature_matrix,coefficients)
    # Compute P(y_i = +1 | x_i, w) using the link function
    predictions = 1/(1+np.exp(-score)) #SIGMOID
    # return predictions
    return predictions

In [ ]:
#derivative is required for the hill-climbing process
def feature_derivative(errors, feature_matrix, feature_no, coefficient, feature_w0, l2_penalty):     
    # Compute the dot product of errors and feature
    feature = feature_matrix[:,feature_no]
    derivative = np.dot(errors, np.transpose(feature))
    
    if not feature_w0:
        derivative = derivative - 2*l2_penalty*coefficient
    
    # Return the derivative
    return derivative

In [ ]:
#Log-likelihood shows how well the model performs for the given coefficients
def compute_log_likelihood(feature_matrix, collision, coefficients, l2_penalty):
    indicator = (collision==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))-l2_penalty*np.sum(coefficients[1:]**2)
        
    return lp

In [ ]:
l2_penalty = 0
lp = compute_log_likelihood(train_feature_matrix,train_collision,initial_coefficients,l2_penalty)
lp

In [ ]:
from math import sqrt
def logistic_regression(feature_matrix, collision, initial_coefficients, step_size, max_iter, l2_penalty):
    lp_store = []
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        
        predictions = predict_probability(feature_matrix, coefficients) #prediction from the sigmoid
        # Compute indicator value for (y_i = +1)
        indicator = (collision==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        
        for k in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            feature_w0 = (k==0)
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            derivative = feature_derivative(errors, feature_matrix, k, coefficients[k], feature_w0, l2_penalty)
            # add the step size times the derivative to the current coefficient
            coefficients[k] = coefficients[k] + step_size*derivative
            
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, collision, coefficients, l2_penalty)
            #print('iteration {0}: log likelihood of observed labels = {1}').format(itr, lp)
            lp_store.append([itr,lp])
    lp_store = np.array(lp_store)
    return coefficients, lp_store

In [ ]:
%matplotlib inline
max_iter = 1000
step_size = [1e-5, 1e-4, 2e-4, 3e-4, 4e-4]
plt.hold(True)
l2_penalty = 0

for x in range(len(step_size)):
    
    optimal_coefficients, lp_store = logistic_regression(train_feature_matrix,
                                                         train_collision, initial_coefficients, 
                                                         step_size[x], max_iter, l2_penalty)
    label_s = str(step_size[x])
    plt.plot(lp_store[:,0],lp_store[:,1], label=label_s)

plt.legend()    
plt.show()

In [ ]:
def test_accuracy(test_feature_matrix, coefficients, test_collisions):

    score = np.dot(test_feature_matrix,coefficients)
    predictions = (score > 0)
    collision = (test_collisions == +1)
    correct_predictions = (predictions == collision)
    coefficients = np.array(coefficients)
    accuracy = float(np.count_nonzero(correct_predictions))/len(collision)
    
    return accuracy

In [ ]:
l2_penalty = [0, 4, 19, 1e2, 1e3, 1e5]
step_size = 3e-4

for p in range(len(l2_penalty)):
    
    optimal_coefficients, lp_store = logistic_regression(train_feature_matrix,
                                                         train_collision, initial_coefficients,
                                                         step_size, max_iter, l2_penalty[p])
    print(optimal_coefficients)
    accuracy = test_accuracy(test_feature_matrix, optimal_coefficients, test_collision)
    
    print('l2_penalty = {0}, accuracy = {1:.2f}').format(l2_penalty[p], accuracy)
    
    

In [ ]:
l2_penalty = 4
step_size = 3e-4
max_iter = 3000
optimal_coefficients, lp_store = logistic_regression(train_feature_matrix, train_collision,
                                                     initial_coefficients, step_size, max_iter,l2_penalty)

In [ ]:
accuracy = test_accuracy(test_feature_matrix, optimal_coefficients, test_collision)
print(accuracy)
accuracy_train = test_accuracy(train_feature_matrix, optimal_coefficients, train_collision)

In [ ]:
print(features)
print(optimal_coefficients)